# MP2

## Some useful resources:
 - [original paper](https://journals.aps.org/pr/abstract/10.1103/PhysRev.46.618)
 - Levine Chapter 16
 - [psi4numpy tutorial](https://github.com/psi4/psi4numpy/blob/master/Tutorials/05_Moller-Plesset/5a_conventional-mp2.ipynb)
 - [Crawdad programming notes](http://sirius.chem.vt.edu/wiki/doku.php?id=crawdad:programming:project4)

# MP2 algorithm
1. The starting point will be the Hartree-Fock wavefunction. 

## Imports

In [ ]:
import numpy as np
import scipy.linalg as spla
import pyscf
import matplotlib.pyplot as plt
import time
%matplotlib notebook

## Specify the molecule

In [ ]:
# start timer
start_time = time.time()
# define molecule
mol = pyscf.gto.M(
    atom="O 0.0000000 0.0000000 0.0000000; H 0.7569685 0.0000000 -0.5858752; H -0.7569685 0.0000000 -0.5858752",
    basis='sto-3g',
    unit="Ang",
    verbose=0,
    symmetry=False,
    spin=0,
    charge=0
)
# get number of atomic orbitals
num_ao = mol.nao_nr()
# get number of electrons
num_elec_alpha, num_elec_beta = mol.nelec
num_elec = num_elec_alpha + num_elec_beta
# get nuclear repulsion energy
E_nuc = mol.energy_nuc()

## Calculate molecular integrals 


Overlap 

$$ S_{\mu\nu} = (\mu|\nu) = \int dr \phi^*_{\mu}(r) \phi_{\nu}(r) $$

Kinetic

$$ T_{\mu\nu} = (\mu\left|-\frac{\nabla}{2}\right|\nu) = \int dr \phi^*_{\mu}(r) \left(-\frac{\nabla}{2}\right) \phi_{\nu}(r) $$

Nuclear Attraction

$$ V_{\mu\nu} = (\mu|r^{-1}|\nu) = \int dr \phi^*_{\mu}(r) r^{-1} \phi_{\nu}(r) $$

Form Core Hamiltonian

$$ H = T + V $$

Two electron integrals

$$ (\mu\nu|\lambda\sigma) = \int dr_1 dr_2 \phi^*_{\mu}(r_1) \phi_{\nu}(r_1) r_{12}^{-1} \phi^*_{\lambda}(r_2) \phi_{\sigma}(r_2) $$


In [ ]:
# calculate overlap integrals
S = mol.intor('cint1e_ovlp_sph')
# calculate kinetic energy integrals
T = mol.intor('cint1e_kin_sph')
# calculate nuclear attraction integrals
V = mol.intor('cint1e_nuc_sph')
# form core Hamiltonian
H = T + V
# calculate two electron integrals
eri = mol.intor('cint2e_sph', aosym='s8')
# since we are using the 8 fold symmetry of the 2 electron integrals
# the functions below will help us when accessing elements
__idx2_cache = {}


def idx2(i, j):
    if (i, j) in __idx2_cache:
        return __idx2_cache[i, j]
    elif i >= j:
        __idx2_cache[i, j] = int(i*(i+1)/2+j)
    else:
        __idx2_cache[i, j] = int(j*(j+1)/2+i)
    return __idx2_cache[i, j]


def idx4(i, j, k, l):
    return idx2(idx2(i, j), idx2(k, l))


print(np.shape(eri))

## Perform Hartree-Fock SCF

In [ ]:
# set inital density matrix to zero
D = np.zeros((num_ao, num_ao))

# 2 helper functions for printing during SCF


def print_start_iterations():
    print("{:^79}".format("{:>4}  {:>11}  {:>11}  {:>11}  {:>11}".format(
        "Iter", "Time(s)", "RMSC DM", "delta E", "E_elec")))
    print("{:^79}".format("{:>4}  {:>11}  {:>11}  {:>11}  {:>11}".format(
        "****", "*******", "*******", "*******", "******")))


def print_iteration(iteration_num, iteration_start_time, iteration_end_time, iteration_rmsc_dm, iteration_E_diff, E_elec):
    print("{:^79}".format("{:>4d}  {:>11f}  {:>.5E}  {:>.5E}  {:>11f}".format(iteration_num,
                                                                              iteration_end_time - iteration_start_time, iteration_rmsc_dm, iteration_E_diff, E_elec)))


# set stopping criteria
iteration_max = 100
convergence_E = 1e-9
convergence_DM = 1e-5
# loop variables
iteration_num = 0
E_scf_total = 0
E_scf_elec = 0.0
iteration_E_diff = 0.0
iteration_rmsc_dm = 0.0
converged = False
exceeded_iterations = False

print_start_iterations()
while (not converged and not exceeded_iterations):
    # store last iteration and increment counters
    iteration_start_time = time.time()
    iteration_num += 1
    E_elec_last = E_scf_elec
    D_last = np.copy(D)
    # form G matrix
    G = np.zeros((num_ao, num_ao))
    for i in range(num_ao):
        for j in range(num_ao):
            for k in range(num_ao):
                for l in range(num_ao):
                    G[i, j] += D[k, l] * \
                        ((2.0*(eri[idx4(i, j, k, l)])) -
                         (eri[idx4(i, k, j, l)]))
    # build fock matrix
    F = H + G
    # solve the generalized eigenvalue problem
    E_orbitals, C = spla.eigh(F, S)
    # compute new density matrix
    D = np.zeros((num_ao, num_ao))
    for i in range(num_ao):
        for j in range(num_ao):
            for k in range(num_elec_alpha):
                D[i, j] += C[i, k] * C[j, k]
    # calculate electronic energy
    E_scf_elec = np.sum(np.multiply(D, (H + F)))
    # calculate energy change of iteration
    iteration_E_diff = np.abs(E_scf_elec - E_elec_last)
    # rms change of density matrix
    iteration_rmsc_dm = np.sqrt(np.sum((D - D_last)**2))
    iteration_end_time = time.time()
    print_iteration(iteration_num, iteration_start_time,
                    iteration_end_time, iteration_rmsc_dm, iteration_E_diff, E_scf_elec)
    if(np.abs(iteration_E_diff) < convergence_E and iteration_rmsc_dm < convergence_DM):
        converged = True
    if(iteration_num == iteration_max):
        exceeded_iterations = True

# calculate total energy
E_scf_total = E_scf_elec + E_nuc
print("{:^79}".format("Total HF Energy : {:>11f}".format(E_scf_total)))

# Perform MP2 calculation

## Convert the two-electron integrals from AO basis to the MO basis

$$(pq|rs) = \sum_\mu \sum_\nu \sum_\lambda \sum_\sigma C_\mu^p C_\nu^q
(\mu \nu|\lambda \sigma) C_\lambda^r C_\sigma^s.$$


Attempt to code this conversion below. Although this was introduced previously, we want to remention a note about the electron repulsion integrals. The electron repulsion integrals are stored as vector `eri`. This vector represent what would otherwise be a 4-D tensor. The reason we can store it as a vector is that there is symmetry present for the two-electron integrals and pyscf incorporates that. Thus we access our integrals using a helper function we have written for you, `idx_4` above which will turn the 4 index into the correct index for the pyscf data structure. For example:

To access the specific integral for $(\phi_1\phi_4|\phi_7\phi_9)$, we would use the command
`eri[idx4(1,4,7,9)]`. This will be essential for writing the transofmration inthe code cell below and in calculating the MP2 energy afterwards.



In [ ]:
## place code for two-electron integral conversion here.

### Compute the MP2 Energy
Now we can calculate the MP2 estimation of the correlation energy. 
$$E_{\mathrm{corr(MP2)}}\ =\ \frac{( ia \mid jb ) [ 2 (ia \mid jb ) - ( ib \mid ja )]}{\epsilon_i + \epsilon_j + \epsilon_a - \epsilon_b}$$

Here $i$ and $j$ represent all occupied orbitals, where as $a$ and $b$ will be unoccupied orbitals. 

Remember during this coding step that we are basing our MP2 correction on an RHF calculation and thus there are the same amount of $\alpha$ and $\beta$ electrons.

In [ ]:
#initialize the variable forthe mp2 correlation energy
E_corr_mp2 = 0
# code the equation above and adjust the value of E_corr_mp2


#this will print your E_corr mp2
print("{:^79}".format("Total MP2 correlation energy : {:>11f}".format(E_corr_mp2)))

The correlation energy is very small compared to the total energy, which is generally the case. However, this correlation energy can be very important to describing properties such as dispersion. 

## A comparison with PySCF

In [ ]:
import pyscf
m = pyscf.scf.RHF(mol)
print('E(HF) = %g' % m.kernel())
mp2 = pyscf.mp.MP2(m)
E_corr_mp2_pyscf = mp2.kernel()[0]
print('E(MP2) = {:.9g}'.format(E_corr_mp2_pyscf))

In [ ]:
# comparison from pyscf
E_diff = E_corr_mp2_pyscf - E_corr_mp2
print(E_diff)